In [2]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [3]:
df_trades = pd.read_pickle('BTC_USD_1m_trades.pkl')

In [4]:
pair = 'BTC_USD'

In [5]:
df_raw = pd.read_csv(utils.get_his_data_filename(pair, '1m'))

In [6]:
non_cols = ['open_time', 'close_time','volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [7]:
df_trades['open_time'] = [parse(x) for x in df_trades.open_time]
df_raw['open_time'] = [parse(x) for x in df_raw.open_time]

In [8]:
df_trades['next'] = df_trades['open_time'].shift(-1)

In [9]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(seconds=55)
df_trades['trade_start'] = df_trades.open_time + dt.timedelta(minutes=1)

In [10]:
df_trades[['open_time', 'next', 'trade_end']].head()

,open_time,next,trade_end
5,2025-11-01 00:06:00+00:00,2025-11-01 00:18:00+00:00,2025-11-01 00:18:55+00:00
17,2025-11-01 00:18:00+00:00,2025-11-01 00:27:00+00:00,2025-11-01 00:27:55+00:00
26,2025-11-01 00:27:00+00:00,2025-11-01 00:50:00+00:00,2025-11-01 00:50:55+00:00
49,2025-11-01 00:50:00+00:00,2025-11-01 00:59:00+00:00,2025-11-01 00:59:55+00:00
58,2025-11-01 00:59:00+00:00,2025-11-01 01:19:00+00:00,2025-11-01 01:19:55+00:00


In [15]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [ ]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)

    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction
    
    print('ERROR')


def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0


def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)


def process_s5(s5_df, row):
    result = 0.0
    for index, price in enumerate(s5_df.close.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            #print(f'Signal at {index} {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}')
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, s5_df.close.values, row.ENTRY)
            break
    return result

In [24]:
total = 0
for index, row in df_trades.iterrows():
    s5_data = df_raw[(df_raw.open_time >= row.trade_start) & (df_raw.open_time <= row.trade_end)]
    #print(f'{row.open_time} {signal_text(row.SIGNAL)} {row.ENTRY:.2f} {s5_data.shape}')
    total += process_s5(s5_data, row)
    #if index > 10:
    #    break
print(total)

Signal at 7 109539.58 109527.93 1.00
Signal at 15 109549.05 109543.95 1.00
Signal at 0 109751.98 109751.45 1.00
Signal at 18 109716.58 109723.40 -1.00
Signal at 42 109672.85 109697.79 -1.00
Signal at 1 109616.88 109627.91 -1.00
Signal at 1 109651.53 109669.44 -1.00
Signal at 0 109560.91 109546.01 1.00
Signal at 2 109807.89 109798.27 1.00
Signal at 0 110081.52 109981.67 1.00
Signal at 0 110124.96 110114.87 1.00
Signal at 3 110346.00 110332.38 1.00
Signal at 1 110461.76 110384.50 1.00
Signal at 0 110555.03 110511.59 1.00
Signal at 2 110187.08 110256.02 -1.00
Signal at 0 110212.84 110228.63 -1.00
Signal at 1 110296.13 110252.26 1.00
Signal at 18 110100.00 110081.78 1.00
Signal at 0 110056.53 110109.38 -1.00
Signal at 0 109995.61 109981.85 1.00
Signal at 1 110126.89 110098.64 1.00
Signal at 0 110094.14 110104.53 -1.00
Signal at 0 110074.01 110094.68 -1.00
Signal at 2 110194.58 110162.77 1.00
Signal at 2 110185.01 110187.90 -1.00
Signal at 10 110254.55 110240.67 1.00
Signal at 0 110049.98 1